In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("/home/xinyang/Kaggle/TRD/TRD_Dalyr.csv",encoding='utf-16',sep='\t')
df2=pd.read_csv("/home/xinyang/Kaggle/TRD/TRD_Dalyr1.csv",encoding='utf-16',sep='\t')
df_con1=pd.concat([df,df2])
df4=pd.read_csv("/home/xinyang/Kaggle/TRD2/TRD_Dalyr.csv",encoding='utf-16',sep='\t')
df5=pd.read_csv("/home/xinyang/Kaggle/TRD2/TRD_Dalyr1.csv",encoding='utf-16',sep='\t')
df6=pd.read_csv("/home/xinyang/Kaggle/TRD2/TRD_Dalyr2.csv",encoding='utf-16',sep='\t')
df_con2=pd.concat([df4,df5,df6])
df7=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr.csv",encoding='utf-16',sep='\t')
df8=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr1.csv",encoding='utf-16',sep='\t')
df9=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr2.csv",encoding='utf-16',sep='\t')
df10=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr3.csv",encoding='utf-16',sep='\t')
df_con3=pd.concat([df7,df8,df9,df10])
df_con=pd.concat([df_con1,df_con2,df_con3])

/home/xinyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """
/home/xinyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [2]:
df3=pd.read_csv("/home/xinyang/Kaggle/zcfg/FS_Combas.csv",encoding='utf-16',sep='\t')
df3=df3.rename(columns={'Accper':'Trddt'})
df3['islong'] = 1

In [3]:
# Specify the Stkcd number. Use 1 as an example here. 
df3_1 = df3[df3.Stkcd==1]
df_con_1 = df_con[df_con.Stkcd == 1]
df_merge_1 = df3_1.merge(df_con_1,on=['Stkcd','Trddt'],how='outer')

In [4]:
df_merge_1.sort_values('Trddt',inplace=True)
df_merge_1 = df_merge_1.reset_index()
df_merge_1 = df_merge_1.drop('index',1)

In [5]:
a = df_merge_1.loc[0,'Typrep':'islong']
for i in range(3465):
    if df_merge_1.loc[i,'islong'] == 1:
        a = df_merge_1.loc[i,'Typrep':'islong']
    else:
        df_merge_1.loc[i,'Typrep':'islong'] = a

In [6]:
df_merge_1.dropna(axis=1,how='all')
df_merge_1 = df_merge_1.fillna(0)
df_merge_1 = df_merge_1.loc[:, (df_merge_1 != 0).any(axis=0)]

In [7]:
df1 = df_merge_1['Dretnd']
#df_merge_1 = df_merge_1.drop(['Typrep','islong','Trddt','Capchgdt'],axis =1)
num_1 = (df_merge_1.loc[:,'A0b1103000':'A004000000']).shape[1]
num_2 = (df_merge_1.loc[:,'Adjprcnd':'Trdsta']).shape[1]

In [8]:
num_2

15

In [9]:
# Append the dataframe that 10-day's data is merged to one row
df_zcfg = df_merge_1.loc[:,'A0b1103000':'A004000000']
df_rgg = df_merge_1.loc[:,'Adjprcnd':'Trdsta']
for i in range(9):
    columns = []
    for j in range(num_1):
        columns.append(str(i)+'_'+str(j))
    df = df_merge_1.iloc[(i+1):,1:(num_1+1)].append(df_merge_1.iloc[:i,1:(num_1+1)])
    df.columns= columns
    df = df.reset_index(drop=True)
    df_zcfg = pd.concat([df_zcfg,df],axis=1) 
    columns = []
    for j in range(num_2):
        columns.append(str(i)+'_'+str(j))
    df = df_merge_1.iloc[(i+1):,(num_1+1):(num_1+num_2+1)].append(df_merge_1.iloc[:(i+1),(num_1+1):(num_1+num_2+1)])
    df.columns= columns
    df = df.reset_index(drop=True)
    df_rgg = pd.concat([df_rgg,df],axis=1) 

In [10]:
df_zcfg.shape

(3465, 640)

In [11]:
df_rgg.shape

(3465, 150)

In [12]:
df1.iloc[:(-11)].append(df1.iloc[:11])
df1 = df1.reset_index(drop=True)
#df_merge_1['goal'] = df1

In [13]:
df_zcfg = df_zcfg.iloc[:(-11),:]
df_rgg = df_rgg.iloc[:(-11),:]

In [14]:
df_zcfg = df_zcfg.reset_index(drop=True)
df_rgg = df_rgg.reset_index(drop=True)
import featuretools as ft
# Create es
df_zcfg = df_zcfg.reset_index()
df_rgg = df_rgg.reset_index()
es1 = ft.EntitySet(id="zcfg")
es1 =  es1.entity_from_dataframe(entity_id="zcfg",dataframe=df_zcfg,index="index")
es2 = ft.EntitySet(id="rgg")
es2 =  es2.entity_from_dataframe(entity_id="rgg",dataframe=df_rgg,index="index")

In [15]:
feature_matrix1, feature_defs1 = ft.dfs(entityset=es1, 
                                       target_entity="zcfg",
                                       agg_primitives=["max","median","time_since_last","min","avg_time_between",
                                                      "all","mode","n_most_common","any","mean","time_since_first"
                                                      ,"percent_true","last","sum","num_true","trend","std","count"
                                                      ,"num_unique","skew"],
                                      max_depth = 1)

In [16]:
feature_matrix2, feature_defs2 = ft.dfs(entityset=es2, 
                                       target_entity="rgg",
                                       agg_primitives=["max","median","time_since_last","min","avg_time_between",
                                                      "all","mode","n_most_common","any","mean","time_since_first"
                                                      ,"percent_true","last","sum","num_true","trend","std","count"
                                                      ,"num_unique","skew"],
                                      max_depth = 1)

In [17]:
df_input = pd.concat([feature_matrix1,feature_matrix2],axis=1) 
df_input['goal'] = df1

In [18]:
df_input.shape

(3454, 848)